## Updated features that helped in increasing precision@top5% from 0.27 to 0.39

In [ ]:

import pandas as pd
import numpy as np

df = pd.read_csv("../data/processed/features_processed.csv")


df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

df = df.sort_values(["user_id", "timestamp"]).reset_index(drop=True)

print("✅ Data loaded and sorted:", df.shape)


df["first_txn_time"] = df.groupby("user_id")["timestamp"].transform("min")
df["days_since_first_txn"] = (df["timestamp"] - df["first_txn_time"]).dt.days.fillna(0)


df["txn_amount_ratio_user_avg"] = df["amount"] / (df["user_avg_amount"] + 1)
df["txn_amount_ratio_user_avg"].replace([np.inf, -np.inf], 0, inplace=True)

df["is_night"] = df["hour"].apply(lambda h: 1 if h in [0, 1, 2, 3, 4, 5] else 0)


df["user_fraud_rate"] = (
    df.groupby("user_id")["is_fraud"]
    .transform(lambda x: x.shift().expanding().mean())
)
df["user_fraud_rate"] = df["user_fraud_rate"].fillna(0)


merchant_users = df.groupby("merchant_id")["user_id"].nunique()
merchant_txns = df.groupby("merchant_id")["transaction_id"].count()
merchant_overlap = (merchant_users / (merchant_txns + 1)).to_dict()
df["merchant_user_overlap"] = df["merchant_id"].map(merchant_overlap)


device_counts = df["device_type"].value_counts().to_dict()
df["device_txn_count"] = df["device_type"].map(device_counts)

df["rolling_fraud_rate_user_7d"] = 0.0

for uid, group in df.groupby("user_id"):
    idx = group.index
    rolled = (
        group.set_index("timestamp")["is_fraud"]
        .rolling("7D").mean()
        .fillna(0)
        .values
    )
    df.loc[idx, "rolling_fraud_rate_user_7d"] = rolled


df.replace([np.inf, -np.inf], 0, inplace=True)
df.fillna(0, inplace=True)


df.drop(columns=["first_txn_time"], inplace=True, errors="ignore")

cols_to_encode = [c for c in ["transaction_type", "device_type", "day_of_week"] if c in df.columns]
print("Encoding these columns:", cols_to_encode)

df = pd.get_dummies(df, columns=cols_to_encode, drop_first=True)


id_cols = [c for c in ["transaction_id", "user_id", "merchant_id", "location"] if c in df.columns]
print("Dropping ID columns:", id_cols)
df = df.drop(id_cols, axis=1)


df = df.fillna(0)


for col in ["amount", "user_txn_sum", "weighted_amount"]:
    if col in df.columns:
        df[col] = np.log1p(df[col])

bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

output_path = "../data/processed/features_enriched.csv"
df.to_csv(output_path, index=False)

print(" Saved enriched dataset ->", output_path)
print(" New dataset shape:", df.shape)


✅ Data loaded and sorted: (10000, 34)
Encoding these columns: ['transaction_type', 'device_type', 'day_of_week']
Dropping ID columns: ['transaction_id', 'user_id', 'merchant_id', 'location']
✅ Saved enriched dataset -> ../data/processed/features_enriched.csv
🧾 New dataset shape: (10000, 45)


Trying new features so that our Precision@Top5% go above 60%

In [ ]:


import pandas as pd
import numpy as np


df = pd.read_csv("../data/processed/features_processed.csv")

# Convert timestamp and sort
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.sort_values(["user_id", "timestamp"]).reset_index(drop=True)
print("✅ Data loaded and sorted:", df.shape)


# 1️⃣ Days since first transaction
df["first_txn_time"] = df.groupby("user_id")["timestamp"].transform("min")
df["days_since_first_txn"] = (df["timestamp"] - df["first_txn_time"]).dt.days.fillna(0)

# 2️⃣ Transaction Amount Ratio to User Average
df["txn_amount_ratio_user_avg"] = df["amount"] / (df["user_avg_amount"] + 1)
df["txn_amount_ratio_user_avg"].replace([np.inf, -np.inf], 0, inplace=True)

# 3️⃣ Is Night Transaction (12AM–5AM)
df["is_night"] = df["hour"].apply(lambda h: 1 if h in [0, 1, 2, 3, 4, 5] else 0)

# 4️⃣ User Historical Fraud Rate
df["user_fraud_rate"] = df.groupby("user_id")["is_fraud"].transform(
    lambda x: x.shift().expanding().mean()
).fillna(0)

# 5️⃣ Merchant-User Overlap Ratio
merchant_users = df.groupby("merchant_id")["user_id"].nunique()
merchant_txns = df.groupby("merchant_id")["transaction_id"].count()
merchant_overlap = (merchant_users / (merchant_txns + 1)).to_dict()
df["merchant_user_overlap"] = df["merchant_id"].map(merchant_overlap)

# 6️⃣ Device Transaction Count
device_counts = df["device_type"].value_counts().to_dict()
df["device_txn_count"] = df["device_type"].map(device_counts)

# 7️⃣ Rolling Fraud Rate (Past 7 Days per User) – Safe version
df["rolling_fraud_rate_user_7d"] = 0.0
for uid, group in df.groupby("user_id"):
    idx = group.index
    rolled = (
        group.set_index("timestamp")["is_fraud"]
        .rolling("7D").mean()
        .fillna(0)
        .values
    )
    df.loc[idx, "rolling_fraud_rate_user_7d"] = rolled


# 🔹 A. Burst / Velocity Features
df["time_diff_sec"] = df.groupby("user_id")["timestamp"].diff().dt.total_seconds().fillna(99999)

# Rolling count of transactions in last 5 minutes (per user)
df["txns_in_last_5min"] = 0.0
for uid, group in df.groupby("user_id"):
    idx = group.index
    rolled = (
        group.set_index("timestamp")["transaction_id"]
        .rolling("5min").count()
        .fillna(0)
        .values
    )
    df.loc[idx, "txns_in_last_5min"] = rolled

# Average time gap between last 3 transactions
df["avg_time_gap_prev3"] = (
    df.groupby("user_id")["time_diff_sec"].transform(lambda x: x.rolling(3).mean())
)
df["avg_time_gap_prev3"] = df["avg_time_gap_prev3"].fillna(df["time_diff_sec"].median())

# 🔹 B. Device & Location Consistency
df["user_device_count"] = df.groupby("user_id")["device_type"].transform("nunique")
df["user_location_count"] = df.groupby("user_id")["location"].transform("nunique")
df["device_location_ratio"] = df["user_device_count"] / (df["user_location_count"] + 1)

# 🔹 C. Merchant Risk Propagation
# Historical fraud rate per merchant
merchant_fraud_rate = df.groupby("merchant_id")["is_fraud"].mean().to_dict()
df["merchant_past_fraud_rate"] = df["merchant_id"].map(merchant_fraud_rate)

# 🔹 D. Recent Activity Weight (exponential decay)
# Assign higher weight to recent transactions
df["txn_order"] = df.groupby("user_id").cumcount()
df["recent_txn_weight"] = np.exp(-0.01 * df["txn_order"])  # more recent = higher weight

# 🔹 E. Peer-based Spending Ratio (Deviation from median user)
user_median_amount = df.groupby("user_id")["amount"].transform("median") + 1
df["peer_spend_ratio"] = df["amount"] / user_median_amount
df["peer_spend_ratio"].replace([np.inf, -np.inf], 0, inplace=True)

# ================================================================
# STEP 4: CLEANUP
# ================================================================
df.replace([np.inf, -np.inf], 0, inplace=True)
df.fillna(0, inplace=True)

# Drop helper columns
df.drop(columns=["first_txn_time"], inplace=True, errors="ignore")

# ================================================================
# STEP 5: ENCODING & FINAL CLEANUP
# ================================================================
cols_to_encode = [c for c in ["transaction_type", "device_type", "day_of_week"] if c in df.columns]
print("Encoding these columns:", cols_to_encode)
df = pd.get_dummies(df, columns=cols_to_encode, drop_first=True)

# Drop ID columns for model training
id_cols = [c for c in ["transaction_id", "user_id", "merchant_id", "location"] if c in df.columns]
print("Dropping ID columns:", id_cols)
df = df.drop(id_cols, axis=1)

# Convert bools to ints
bool_cols = df.select_dtypes(include="bool").columns
df[bool_cols] = df[bool_cols].astype(int)

# Log-transform skewed monetary features
for col in ["amount", "user_txn_sum", "weighted_amount"]:
    if col in df.columns:
        df[col] = np.log1p(df[col])

output_path = "../data/processed/features_enriched_v2.csv"
df.to_csv(output_path, index=False)
print("✅ Saved enriched dataset ->", output_path)
print("🧾 Final shape:", df.shape)


FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/features_processed.csv'

In [4]:
df=pd.read_csv("../data/processed/features_enriched_v2.csv")
df.columns.tolist()


['timestamp',
 'amount',
 'is_fraud',
 'hour',
 'is_weekend',
 'user_txn_count',
 'user_txn_sum',
 'user_avg_amount',
 'user_std_amount',
 'amount_zscore_user',
 'time_diff',
 'txn_velocity',
 'merchant_txn_count',
 'merchant_unique_users',
 'merchant_fraud_rate',
 'device_risk_score',
 'location_risk_score',
 'location_device_interaction',
 'txn_count_1h',
 'txn_count_24h',
 'is_high_amount',
 'is_new_user',
 'is_high_velocity',
 'is_risky_device',
 'is_risky_location',
 'weighted_amount',
 'user_risk_score',
 'days_since_first_txn',
 'txn_amount_ratio_user_avg',
 'is_night',
 'user_fraud_rate',
 'merchant_user_overlap',
 'device_txn_count',
 'rolling_fraud_rate_user_7d',
 'time_diff_sec',
 'txns_in_last_5min',
 'avg_time_gap_prev3',
 'user_device_count',
 'user_location_count',
 'device_location_ratio',
 'merchant_past_fraud_rate',
 'txn_order',
 'recent_txn_weight',
 'peer_spend_ratio',
 'transaction_type_purchase',
 'transaction_type_top-up',
 'transaction_type_transfer',
 'device_